<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#2018-12-18" data-toc-modified-id="2018-12-18-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2018-12-18</a></span><ul class="toc-item"><li><span><a href="#検索組織の機械学習実行基盤-|-リクルートテクノロジーズ　メンバーズブログ" data-toc-modified-id="検索組織の機械学習実行基盤-|-リクルートテクノロジーズ　メンバーズブログ-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>検索組織の機械学習実行基盤 | リクルートテクノロジーズ　メンバーズブログ</a></span><ul class="toc-item"><li><span><a href="#登場キーワードについての調査" data-toc-modified-id="登場キーワードについての調査-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>登場キーワードについての調査</a></span><ul class="toc-item"><li><span><a href="#DRONE-:-CI/CD-のパイプライン処理" data-toc-modified-id="DRONE-:-CI/CD-のパイプライン処理-2.1.1.1"><span class="toc-item-num">2.1.1.1&nbsp;&nbsp;</span>DRONE : CI/CD のパイプライン処理</a></span></li><li><span><a href="#DigDag-:-機械学習処理システム向けのパイプライン処理ツール" data-toc-modified-id="DigDag-:-機械学習処理システム向けのパイプライン処理ツール-2.1.1.2"><span class="toc-item-num">2.1.1.2&nbsp;&nbsp;</span>DigDag : 機械学習処理システム向けのパイプライン処理ツール</a></span></li><li><span><a href="#.dig-ファイルのサンプル集" data-toc-modified-id=".dig-ファイルのサンプル集-2.1.1.3"><span class="toc-item-num">2.1.1.3&nbsp;&nbsp;</span>.dig ファイルのサンプル集</a></span></li></ul></li></ul></li></ul></li></ul></div>

# 概要
機械学習について調べた履歴を記録する。  
Jupyter Notebook の便利な機能は、履歴作成に当たって大変助かる。(Table of Contents の自動生成など)
まずは、細かい整理はせず、片っ端から記録するというスタイルで始めるとしよう。  


# 2018-12-18

## 検索組織の機械学習実行基盤 | リクルートテクノロジーズ　メンバーズブログ
  https://recruit-tech.co.jp/blog/2018/09/10/qass_ml/
  
* 機械学習システムのアーキテクチャ図があるので参考になる。
    * ![アーキテクチャ](https://s3-ap-northeast-1.amazonaws.com/prod-rtc-blog/wp-content/uploads/2018/08/31193348/flya.png)
* ソフトウェア資産の再利用性を高めるため、Dockerを活用している。
    * Google Container Registry (GCR)に、コンテナを登録している。
* Google Kubernetes Engine (GKE)の上に、"DRONE", "DigDag" といったアプリケーションを構築。
* 機械学習処理の最終出力は、Google Cloud Storage(GCS)上に保存。
* 処理フローの共通化のポイント
    1. モデル部分は共通化しているが、
    2. 前処理部分は、サービスごとに開発する
    
### 登場キーワードについての調査

#### DRONE : CI/CD のパイプライン処理

JenkinsやCircle CIと同じカテゴリのCI/CDツールだが、**Gitイベント** をトリガーにして、**Dockerコンテナ**内部でジョブを実行することに特化したツールである。  
Dockerを利用したマイクロサービス化が普及するにつれて、DRONEの普及も進んでいる模様。  

1. 対象Gitリポジトリのルートディレクトリにある **.drone.yml** を読み込んで、パイプライン処理を実行する。
   .drone.ymlの例
   ***
```
pipeline:
  # ステップは上から順に実行されます（順序付きmap）。
  hello:
    # ステップ毎にイメージを指定
    image: alpine:3.6
    # Dockerコンテナ起動時のエントリポイントとしてコマンドが実行されます。
    # 各コマンドは同じシェルで実行されます。
    commands:
      - echo 　"Hello World"
```
   ***

参照: https://engineering.linecorp.com/ja/blog/go-oss-ci-tool-drone-replaces-jenkins/


#### DigDag : 機械学習処理システム向けのパイプライン処理ツール

機械学習処理システムは、処理フローが複雑化しやすく（特に前処理）、技術的負債の利子が危険レベルになる傾向がある。  
処理フローを記述しやすく、把握しやすく管理できるのがDigDagとのこと。  
以下のように、処理フローを階層的に記述しやすい。

![DigDag処理フローイメージ](http://docs.digdag.io/_images/grouping-tasks.png)


yml的な記述をする .dig ファイルに処理フローの定義を記述する。

.dig の例
***
```text
# daily_etl.dig
schedule:
  daily>: 00:05:00
 
+foo:
  _retry: 3
 
  +get_foo_metrics1:
    sh>: SESSION_TIME=${session_time} python elasticsearch_client.py foo metrics1 1day > foo_metrics1_daily.csv
  +vaidate_foo_metrics1:
    sh>: bash data_validator.sh foo_metrics1_daily.csv
  +commit_foo_metrics1:
    sh>: bash commit.sh foo_metrics1_daily.csv
  +get_foo_metrics2:
    sh>: SESSION_TIME=${session_time} python elasticsearch_client.py foo metrics2 1day > foo_metrics2_daily.csv
  +vaidate_foo_metrics2:
    sh>: bash data_validator.sh foo_metrics2_daily.csv
  +commit_foo_metrics2:
    sh>: bash commit.sh foo_metrics2_daily.csv

```
***

学習処理を定義する .dig ファイルの例を示す。
***
**daily_train.dig**
```
# daily_train.dig
schedule:
  daily>: 00:10:00
 
+wait_daily_etl:
  require>: daily_etl
 
+foo:
  +train:
    _parallel: true
 
    +train_foo_metrics1:
      sh>: python train.py foo metrics1 foo_metrics1_daily.csv
    +train_foo_metrics2:
      sh>: python train.py foo metrics2 foo_metrics2_daily.csv
 
  +cross_validation:
    _pararell: true
 
    +cross_validation_foo_metrics1:
      sh>: python cross_validation.py foo metrics1
    +cross_validation_foor_metrics2:
      sh>: python cross_validation.py foo metrics2
 
_error:
  sh>: bash notify.sh
```
***

cross_validation.py は、過学習を起こしていたと判断すると、終了コード 1 を返すプログラムとなっている。  
過学習が起きると \_error が呼び出され、通知処理(notify.sh)が起動するようになっている。  


週次の処理をする場合は、以下のような依存関係を前提とした .digファイルを記述する。  
以下の.digファイルは、 requireで指定された daily_train.dig が7回実行されるのを待ってから、foo 以下の処理が実行される。

***
```
# fine_tuning.dig
schedule:
  weekly>: Mon,01:05:00
 
+wait_daily_train:
  loop>: 7
  _do:
    require>: daily_train
    session_time: ${moment(last_session_time).add(i, 'day').format()}
 
+foo:
  +train:
    _parallel: true
 
    +fine_tuning_foo_metrics1:
      sh>: python fine_tuning.py foo metrics1
    +fine_tuning_foo_metrics2:
      sh>: python fine_tuning.py foo metrics2
```
***


定期的にポーリングして、モデルの性能悪化を検出したら、すかさず通知したいときには、以下のような .digファイルを作ると良いという。  
以下では5分ごとに predict.py を実行し、モデルの性能が悪化して exit_code = 1 となったら、_error がキックされ、通知処理(notify.sh)が実行される
。  

*** 
```
# predict.dig
schedule:
  minutes_interval>: 5
 
+foo:
  +predict:
    _parallel: true
 
    +predict_foo_metrics1:
      sh>: python predict.py foo metrics1
    +predict_foo_metrics2:
      sh>: python predict.py foo metrics2
 
_error:
  sh>: bash notify.sh
```
***

#### .dig ファイルのサンプル集

トレジャーデータの以下のサイトで紹介している。  
https://github.com/treasure-data/digdag/tree/master/examples

##### 参照リンク
* [DigDag公式サイト](http://docs.digdag.io/index.html)
* https://www.lifull.blog/entry/2017/09/20/151600